In [102]:
import SIRD_Model

import numpy as np
import csv
import matplotlib.pyplot as plt
import platform

In [103]:
pathc = "../Data/Italian Data/"
# modify the path syntax based on the current platform
if platform.system() == "Windows":
    pathc.replace("/", "\\")
    
filename = "National Data.csv"

dates, infectRaw, recovRaw, deadRaw = SIRD_Model.loadData(pathc + filename)

In [104]:
import scipy.optimize as opt

#note paramArg is in the format of [q, beta, gamma, nu] [q, b1, b2, b3, b0, gamma, nu]
def errorSIRD(paramArg, pop, I, R, D, lamda, w): #the custom error function for SIRD    
    
    q = paramArg[0]
    b = paramArg[0:4]
    b[0] = paramArg[4] #b=[b0, b1, b2, b3]
    params = paramArg[-3:] #beta0, gamma, nu
    
    #note: beta = b0 / (1 + (I*b1 + D*b2)^b3)
    #use b0 in the place of beta, and multiply first column by 1/(1+(I*b1+D*b2)^b3)
    
    S = q*pop - I - R - D #q*Pop = S + I + R + D
    
    y, A = SIRD_Model.getSIRDMatrices(S, I, R, D)
    
    #transform location where beta is needed and multiply by 1/(1+(I*b1+D*b2)^b3)
    A[:,0,0] = A[:,0,0] / (1 + (I[0:-1]*b[1] + D[0:-1]*b[2])**b[3])
    A[:,1,0] = A[:,1,0] / (1 + (I[0:-1]*b[1] + D[0:-1]*b[2])**b[3])
    
    totalError = 0
    #see paper for optimization function
    T = len(A)
    for t in range(T):
        totalError = totalError + (w**(T - t))*(np.linalg.norm((A[t] @ params) - y[t].transpose(), ord=2)**2)
    
    #return (1.0/T) * np.linalg.norm((A @ params) - y.transpose(), ord=2)**2  + lamda*np.linalg.norm(params, ord=1)
    totalError = (1.0/T)*totalError #divide by timeframe
    totalError = totalError + lamda*np.linalg.norm(params, ord=1) #regularization error
    return totalError



In [105]:
pop = 60000000
numDays = 30 #len(infectRaw) #max days
infect = infectRaw[0:numDays]
recov = recovRaw[0:numDays]
dead = deadRaw[0:numDays]

boundQ = (0,1) #q should be between 0 and 1

#Beta = b0/(1+(I*b1 + D*b2)^b3)
boundB0 = (0,10)
boundB1 = (0,10)
boundB2 = (0,10)
boundB3 = (0,5)

boundG = (0,1) #recovery rate
boundN = (0,1) #death rate

lamda = 10 #regularization weight
wVal = .9 #weight decay

constraints = [boundQ, boundB0, boundB1, boundB2, boundB3, boundG, boundN]
constArg = (pop, infect, recov, dead, lamda, wVal)
initGuess = [.1, .1,.01,.01,1, .1, .01] #init values, q, b0,b1,b2,b3, gamma, nu

#result = opt.shgo(errorSIRD, constraints, args=constArg)
result = opt.fmin(errorSIRD, initGuess, args=constArg)

print(result)

[ 3.38939609  0.4756621  -0.13181003  0.32609584  3.38939609  0.01832469
  0.01606274]


In [241]:
from sklearn import linear_model

def getLinVarsSIRD(paramArg, pop, I, R, D, lamda, w): #calculate the linear vars for the SIRD model, b0, gamma, nu  
    
    q = paramArg[0]
    b1 = paramArg[1]
    b2 = paramArg[2]
    b3 = paramArg[3]
    
    #b0, gamma, nu should be solved for
    
    #note: beta = b0 / (1 + (I*b1 + D*b2)^b3)
    
    S = q*pop - I - R - D #q*Pop = S + I + R + D
    
    nextIterMatrix, sirdMatrix = SIRD_Model.getSIRDMatricesFlat(S,I,R,D)
    #construct y and A, see paper for solving the lasso optimization
    T = int(len(nextIterMatrix)/4)
    y = np.zeros((T*4, 1))
    A = np.zeros((T*4, 3))
    for t in range(T):
        y[4*t+0] = nextIterMatrix[4*t+0] * np.sqrt(w**(T - t))
        y[4*t+1] = nextIterMatrix[4*t+1] * np.sqrt(w**(T - t))
        y[4*t+2] = nextIterMatrix[4*t+2] * np.sqrt(w**(T - t))
        y[4*t+3] = nextIterMatrix[4*t+3] * np.sqrt(w**(T - t))

        #transform location where beta is needed and multiply by 1/(1+(I*b1+D*b2)^b3)
        sirdMatrix[4*t+0] = sirdMatrix[4*t+0] / (1 + (I[t]*b1 + D[t]*b2)**b3)
        sirdMatrix[4*t+0] = sirdMatrix[4*t+1] / (1 + (I[t]*b1 + D[t]*b2)**b3)
        
        A[4*t+0] = sirdMatrix[4*t+0] * np.sqrt(w**(T - t))
        A[4*t+1] = sirdMatrix[4*t+1] * np.sqrt(w**(T - t))
        A[4*t+2] = sirdMatrix[4*t+2] * np.sqrt(w**(T - t))
        A[4*t+3] = sirdMatrix[4*t+3] * np.sqrt(w**(T - t))
            
            
    model = linear_model.Lasso(alpha=lamda, fit_intercept=False, positive=True)
    model.fit(A,y)
    params = model.coef_
    
    totalError = (1.0/T) * np.linalg.norm((A @ params) - y.transpose(), ord=2)**2  + lamda*np.linalg.norm(params, ord=1)
    return list(params)

def gridNonLinVars(paramArg, constraints, varResols, pop, I, R, D, lamda, w): #solve for non linear vars, q, b1, b2, b3
    #note: paramArg = [b0, gamma, nu]
    #b0 = paramArg[0]
    #gamma = paramArg[1]
    #nu = paramArg[2]
    
    #varSteps[:] = constraints[:][0] + (constraints[:][1] - constraints[:][0])/varResols[:]
    varSteps = []
    for i in range(len(constraints)):
        varSteps.append(constraints[i][0] + (constraints[i][1] - constraints[i][0])/varResols[i]) #min + (max - min)/resol
    
    #note beta = b0/(1 + (b1*I + b2*D)^b3)
    #assume starting vals as best starting value
    #minVars = constraints[:][0]
    minVars = []
    for i in range(len(constraints)): #fill minVars with the minimum starting value
        minVars.append((constraints[i][0]))
    
    paramArg = getLinVarsSIRD(minVars, pop, I, R, D, lamda, w)
    minCost = errorSIRD(minVars + paramArg, pop, I, R, D, lamda, w) #the custom error function for SIRD
    
    currVars = minVars
    currCost = minCost
    varIndex = 0 #which var to iterate
    #while the var isn't above it's max
    endLoop = False
    #this could be achieved by using many for loops, but this is a more generalized appraoch
    while(not endLoop): #while varIndex is in range, ends when everything is iterated
        
        paramArg = getLinVarsSIRD(currVars, pop, I, R, D, lamda, w)
        currCost = errorSIRD(currVars + paramArg, pop, I, R, D, lamda, w)
        if(currCost < minCost):
            minCost = currCost
            minVars = currVars
        
        #print("at: ", currVars, currCost)
        
        currVars[varIndex] = currVars[varIndex] + varSteps[varIndex] #iterate variable
        while(currVars[varIndex] > constraints[varIndex][1]): #var value is out of range
            currVars[varIndex] = constraints[varIndex][0] #reset to minimum
            varIndex = varIndex + 1 #move to iterating the next variable
            if(varIndex == len(currVars)): #out of range endLoop
                endLoop = True
                break
            currVars[varIndex] = currVars[varIndex] + varSteps[varIndex] #iterate variable
        varIndex = 0 #go back to iterating firs var
    return minVars

def solveAllVars(initGuess, nonLinConstraints, nonLinResol, pop, I, R, D, lamda, w, iterations = 3):
    nonLinVars = initGuess[1:5]
    nonLinVars[0] = initGuess[0] #nonLinVar = [q, b1, b2, b3]
    
    linVars = initGuess[5:]
    linVars[0] = initGuess[1] #linVars = [b0, gamma, nu]
    
    for i in range(iterations): #go back and forth solving for linear and nonlinear variables
        nonLinVars = gridNonLinVars(linVars, nonLinConstraints, nonLinResol, pop, I, R, D, lamda, w)
        print("testing")
        linVars = getLinVarsSIRD(nonLinVars, pop, I, R, D, lamda, w)
        
        print("Iteration: ", i)
        print("q:  ", nonLinVars[0])
        print("b0: ", linVars[0])
        print("b1: ", nonLinVars[1])
        print("b2: ", nonLinVars[2])
        print("b3: ", nonLinVars[3])
        print("g:  ", linVars[1])
        print("nu: ", linVars[2])
        print("cost: ", errorSIRD(nonLinVars + linVars, pop, I, R, D, lamda, w))
        print() #spacer
        
    finalVars = []
    finalVars.append(nonLinVars[0]) #q
    finalVars.append(linVars[0]) #b0
    finalVars.append(nonLinVars[1]) #b1
    finalVars.append(nonLinVars[2]) #b2
    finalVars.append(nonLinVars[3]) #b3
    finalVars.append(linVars[1]) #gamma
    finalVars.append(linVars[2]) #nu
    return finalVars
    

In [242]:
pop = 60000000
numDays = 30 #len(infectRaw) #max days
infect = infectRaw[0:numDays]
recov = recovRaw[0:numDays]
dead = deadRaw[0:numDays]

boundQ = (.01,.2) #q should be between 0 and 1

#Beta = b0/(1+(I*b1 + D*b2)^b3)
#boundB0 = (0,10)
boundB1 = (0,.0001)
boundB2 = (0,0)
boundB3 = (1,1)

#boundG = (0,1) #recovery rate
#boundN = (0,1) #death rate

lamda = 10 #regularization weight
wVal = 1 #weight decay, when w is 

constraints = [boundQ, boundB1, boundB2, boundB3]
varResol = [2,50,1,1]

constArg = (pop, infect, recov, dead, lamda, wVal)
initGuess = [.1, .1,.01,.01,1, .1, .01] #init values, q, b0,b1,b2,b3, gamma, nu

#gridNonLinVars([.1,.1,.1], [boundQ, boundB1, boundB2, boundB3], [3,10,10,1], *constArg)
#gridNonLinVars([.1,.1,.1], [(0,3), (0,3)], [3,3], *constArg)

finalVars = solveAllVars(initGuess, constraints, varResol, *constArg, iterations = 1)

#beta = b0 / 1 + (I*b1+D*b2)**b3
betaVars = finalVars[1:5]
beta = betaVars[0] / (1 + (infect*betaVars[1] + dead*betaVars[2])**betaVars[3])

fig, ax = plt.subplots(figsize=(18,8))
ax.plot(beta, color='orange', label='Transmission Rate')

at:  [0.01, 0, 0, 1] 15324440.500340188
at:  [0.11499999999999999, 0, 0, 1] 15313995.341327846
at:  [0.01, 2e-06, 0, 1] 14804798.885699412
at:  [0.11499999999999999, 2e-06, 0, 1] 14796492.670768213
at:  [0.01, 4e-06, 0, 1] 14347881.278817775
at:  [0.11499999999999999, 4e-06, 0, 1] 14341761.952399159
at:  [0.01, 6e-06, 0, 1] 13951635.883705568
at:  [0.11499999999999999, 6e-06, 0, 1] 13947993.277215214
at:  [0.01, 8e-06, 0, 1] 13611483.424553746
at:  [0.11499999999999999, 8e-06, 0, 1] 13610637.662092129
at:  [0.01, 9.999999999999999e-06, 0, 1] 13321876.406016938
at:  [0.11499999999999999, 9.999999999999999e-06, 0, 1] 13324092.31632565
at:  [0.01, 1.1999999999999999e-05, 0, 1] 13077120.979038883
at:  [0.11499999999999999, 1.1999999999999999e-05, 0, 1] 13082581.909603702
at:  [0.01, 1.3999999999999998e-05, 0, 1] 12871790.867029486
at:  [0.11499999999999999, 1.3999999999999998e-05, 0, 1] 12880598.333100554
at:  [0.01, 1.6e-05, 0, 1] 12700915.09510533
at:  [0.11499999999999999, 1.6e-05, 0, 1

at:  [0.01, 5.600000000000002e-05, 0.0, 1] 12422297.565853836
at:  [0.11499999999999999, 5.600000000000002e-05, 0.0, 1] 12475113.662218558
at:  [0.01, 5.800000000000002e-05, 0.0, 1] 12466445.333947437
at:  [0.11499999999999999, 5.800000000000002e-05, 0.0, 1] 12519918.046194393
at:  [0.01, 6.000000000000002e-05, 0.0, 1] 12511943.090540456
at:  [0.11499999999999999, 6.000000000000002e-05, 0.0, 1] 12565981.13699619
at:  [0.01, 6.200000000000002e-05, 0.0, 1] 12558563.431023108
at:  [0.11499999999999999, 6.200000000000002e-05, 0.0, 1] 12613080.932161437
at:  [0.01, 6.400000000000001e-05, 0.0, 1] 12606105.83038637
at:  [0.11499999999999999, 6.400000000000001e-05, 0.0, 1] 12661022.110046895
at:  [0.01, 6.6e-05, 0.0, 1] 12654393.381457578
at:  [0.11499999999999999, 6.6e-05, 0.0, 1] 12709632.689798247
at:  [0.01, 6.8e-05, 0.0, 1] 12703269.959193062
at:  [0.11499999999999999, 6.8e-05, 0.0, 1] 12758761.2190134
at:  [0.01, 7e-05, 0.0, 1] 12752597.75101549
at:  [0.11499999999999999, 7e-05, 0.0, 1] 

at:  [0.01, 1.3999999999999998e-05, 0.0, 1] 12871790.867029486
at:  [0.11499999999999999, 1.3999999999999998e-05, 0.0, 1] 12880598.333100554
at:  [0.01, 1.6e-05, 0.0, 1] 12700915.09510533
at:  [0.11499999999999999, 1.6e-05, 0.0, 1] 12713098.98466501
at:  [0.01, 1.8e-05, 0.0, 1] 12560043.225688234
at:  [0.11499999999999999, 1.8e-05, 0.0, 1] 12575575.006045425
at:  [0.01, 2e-05, 0.0, 1] 12445245.83423754
at:  [0.11499999999999999, 2e-05, 0.0, 1] 12464051.878344743
at:  [0.01, 2.2000000000000003e-05, 0.0, 1] 12353083.284231126
at:  [0.11499999999999999, 2.2000000000000003e-05, 0.0, 1] 12375055.899670849
at:  [0.01, 2.4000000000000004e-05, 0.0, 1] 12280559.710970972
at:  [0.11499999999999999, 2.4000000000000004e-05, 0.0, 1] 12305567.249086518
at:  [0.01, 2.6000000000000005e-05, 0.0, 1] 12225073.893161558
at:  [0.11499999999999999, 2.6000000000000005e-05, 0.0, 1] 12252968.054165954
at:  [0.01, 2.8000000000000006e-05, 0.0, 1] 12184370.424328463
at:  [0.11499999999999999, 2.8000000000000006e-

at:  [0.01, 7.199999999999999e-05, 0.0, 1] 12802255.10220303
at:  [0.11499999999999999, 7.199999999999999e-05, 0.0, 1] 12858054.490532946
at:  [0.01, 7.399999999999998e-05, 0.0, 1] 12852134.632934602
at:  [0.11499999999999999, 7.399999999999998e-05, 0.0, 1] 12907998.333236951
at:  [0.01, 7.599999999999998e-05, 0.0, 1] 12902141.589999884
at:  [0.11499999999999999, 7.599999999999998e-05, 0.0, 1] 12958014.812758448
at:  [0.01, 7.799999999999997e-05, 0.0, 1] 12952192.401587605
at:  [0.11499999999999999, 7.799999999999997e-05, 0.0, 1] 13008023.87224932
at:  [0.01, 7.999999999999997e-05, 0.0, 1] 13002213.408133285
at:  [0.11499999999999999, 7.999999999999997e-05, 0.0, 1] 13057955.163841061
at:  [0.01, 8.199999999999996e-05, 0.0, 1] 13052139.75066018
at:  [0.11499999999999999, 8.199999999999996e-05, 0.0, 1] 13107746.942455456
at:  [0.01, 8.399999999999995e-05, 0.0, 1] 13101914.367895069
at:  [0.11499999999999999, 8.399999999999995e-05, 0.0, 1] 13157345.092551418
at:  [0.01, 8.599999999999995e

at:  [0.11499999999999999, 2.2000000000000003e-05, 0.0, 1] 12375055.899670849
at:  [0.01, 2.4000000000000004e-05, 0.0, 1] 12280559.710970972
at:  [0.11499999999999999, 2.4000000000000004e-05, 0.0, 1] 12305567.249086518
at:  [0.01, 2.6000000000000005e-05, 0.0, 1] 12225073.893161558
at:  [0.11499999999999999, 2.6000000000000005e-05, 0.0, 1] 12252968.054165954
at:  [0.01, 2.8000000000000006e-05, 0.0, 1] 12184370.424328463
at:  [0.11499999999999999, 2.8000000000000006e-05, 0.0, 1] 12214992.597713526
at:  [0.01, 3.0000000000000008e-05, 0.0, 1] 12156494.574858468
at:  [0.11499999999999999, 3.0000000000000008e-05, 0.0, 1] 12189680.493848775
at:  [0.01, 3.2000000000000005e-05, 0.0, 1] 12139752.144764733
at:  [0.11499999999999999, 3.2000000000000005e-05, 0.0, 1] 12175335.496097513
at:  [0.01, 3.4000000000000007e-05, 0.0, 1] 12132673.646252966
at:  [0.11499999999999999, 3.4000000000000007e-05, 0.0, 1] 12170488.77823296
at:  [0.01, 3.600000000000001e-05, 0.0, 1] 12133983.153881831
at:  [0.1149999

...

at:  [0.01, 5.800000000000002e-05, 0.0, 1] 12466445.333947437
at:  [0.11499999999999999, 5.800000000000002e-05, 0.0, 1] 12519918.046194393
at:  [0.01, 6.000000000000002e-05, 0.0, 1] 12511943.090540456
at:  [0.11499999999999999, 6.000000000000002e-05, 0.0, 1] 12565981.13699619
at:  [0.01, 6.200000000000002e-05, 0.0, 1] 12558563.431023108
at:  [0.11499999999999999, 6.200000000000002e-05, 0.0, 1] 12613080.932161437
at:  [0.01, 6.400000000000001e-05, 0.0, 1] 12606105.83038637
at:  [0.11499999999999999, 6.400000000000001e-05, 0.0, 1] 12661022.110046895
at:  [0.01, 6.6e-05, 0.0, 1] 12654393.381457578
at:  [0.11499999999999999, 6.6e-05, 0.0, 1] 12709632.689798247
at:  [0.01, 6.8e-05, 0.0, 1] 12703269.959193062
at:  [0.11499999999999999, 6.8e-05, 0.0, 1] 12758761.2190134
at:  [0.01, 7e-05, 0.0, 1] 12752597.75101549
at:  [0.11499999999999999, 7e-05, 0.0, 1] 12808274.30961631
at:  [0.01, 7.199999999999999e-05, 0.0, 1] 12802255.10220303
at:  [0.11499999999999999, 7.199999999999999e-05, 0.0, 1] 12

at:  [0.11499999999999999, 2.2000000000000003e-05, 0.0, 1] 12375055.899670849
at:  [0.01, 2.4000000000000004e-05, 0.0, 1] 12280559.710970972
at:  [0.11499999999999999, 2.4000000000000004e-05, 0.0, 1] 12305567.249086518
at:  [0.01, 2.6000000000000005e-05, 0.0, 1] 12225073.893161558
at:  [0.11499999999999999, 2.6000000000000005e-05, 0.0, 1] 12252968.054165954
at:  [0.01, 2.8000000000000006e-05, 0.0, 1] 12184370.424328463
at:  [0.11499999999999999, 2.8000000000000006e-05, 0.0, 1] 12214992.597713526
at:  [0.01, 3.0000000000000008e-05, 0.0, 1] 12156494.574858468
at:  [0.11499999999999999, 3.0000000000000008e-05, 0.0, 1] 12189680.493848775
at:  [0.01, 3.2000000000000005e-05, 0.0, 1] 12139752.144764733
at:  [0.11499999999999999, 3.2000000000000005e-05, 0.0, 1] 12175335.496097513
at:  [0.01, 3.4000000000000007e-05, 0.0, 1] 12132673.646252966
at:  [0.11499999999999999, 3.4000000000000007e-05, 0.0, 1] 12170488.77823296
at:  [0.01, 3.600000000000001e-05, 0.0, 1] 12133983.153881831
at:  [0.1149999

at:  [0.01, 8.199999999999996e-05, 0.0, 1] 13052139.75066018
at:  [0.11499999999999999, 8.199999999999996e-05, 0.0, 1] 13107746.942455456
at:  [0.01, 8.399999999999995e-05, 0.0, 1] 13101914.367895069
at:  [0.11499999999999999, 8.399999999999995e-05, 0.0, 1] 13157345.092551418
at:  [0.01, 8.599999999999995e-05, 0.0, 1] 13151487.160570404
at:  [0.11499999999999999, 8.599999999999995e-05, 0.0, 1] 13206702.270609366
at:  [0.01, 8.799999999999994e-05, 0.0, 1] 13200814.202820461
at:  [0.11499999999999999, 8.799999999999994e-05, 0.0, 1] 13255777.148810692
at:  [0.01, 8.999999999999994e-05, 0.0, 1] 13249857.070622036
at:  [0.11499999999999999, 8.999999999999994e-05, 0.0, 1] 13304533.744524091
at:  [0.01, 9.199999999999993e-05, 0.0, 1] 13298582.24171123
at:  [0.11499999999999999, 9.199999999999993e-05, 0.0, 1] 13352940.833840366
at:  [0.01, 9.399999999999993e-05, 0.0, 1] 13346960.563480001
at:  [0.11499999999999999, 9.399999999999993e-05, 0.0, 1] 13400971.423524326
at:  [0.01, 9.599999999999992

at:  [0.11499999999999999, 3.600000000000001e-05, 0.0, 1] 12173867.074828876
at:  [0.01, 3.800000000000001e-05, 0.0, 1] 12142571.755311575
at:  [0.11499999999999999, 3.800000000000001e-05, 0.0, 1] 12184365.589989068
at:  [0.01, 4.000000000000001e-05, 0.0, 1] 12157474.297365868
at:  [0.11499999999999999, 4.000000000000001e-05, 0.0, 1] 12201024.307590492
at:  [0.01, 4.200000000000001e-05, 0.0, 1] 12177849.690517383
at:  [0.11499999999999999, 4.200000000000001e-05, 0.0, 1] 12223007.930989882
at:  [0.01, 4.400000000000001e-05, 0.0, 1] 12202963.819025557
at:  [0.11499999999999999, 4.400000000000001e-05, 0.0, 1] 12249588.71854243
at:  [0.01, 4.6000000000000014e-05, 0.0, 1] 12232175.26874082
at:  [0.11499999999999999, 4.6000000000000014e-05, 0.0, 1] 12280131.657273838
at:  [0.01, 4.8000000000000015e-05, 0.0, 1] 12264922.747792887
at:  [0.11499999999999999, 4.8000000000000015e-05, 0.0, 1] 12314082.099671088
at:  [0.01, 5.0000000000000016e-05, 0.0, 1] 12300714.48727326
at:  [0.11499999999999999

KeyboardInterrupt: 

In [229]:
print(beta)

[0.01853817 0.01853817 0.01853817 0.01853817 0.01853817 0.01853817
 0.01853817 0.01853817 0.01853817 0.01853817 0.01853817 0.01853817
 0.01853817 0.01853817 0.01853817 0.01853817 0.01853817 0.01853817
 0.01853817 0.01853817 0.01853817 0.01853817 0.01853817 0.01853817
 0.01853817 0.01853817 0.01853817 0.01853817 0.01853817 0.01853817]


In [236]:
paramArg=[finalVars[1], finalVars[-2], finalVars[-1]]
currVars = [.01, 0.0001, 0,1]
paramArg = getLinVarsSIRD(currVars, *constArg)

#SIRD_Model.calculateAverageParams(infect, recov, dead, pop, , graph=True, graphVals=[0,1,0,1])

constArg = (pop, infect, recov, dead, lamda, 1)

currCost = errorSIRD(currVars + paramArg, *constArg)
print(15324440.500340188)
print(currCost)

15324440.500340188
13489778.893064868


In [243]:
for v1 in range(5):
    for v2 in range(5):
        for v3 in range(5):
            print(v1,v2,v3)

0 0 0
0 0 1
0 0 2
0 0 3
0 0 4
0 1 0
0 1 1
0 1 2
0 1 3
0 1 4
0 2 0
0 2 1
0 2 2
0 2 3
0 2 4
0 3 0
0 3 1
0 3 2
0 3 3
0 3 4
0 4 0
0 4 1
0 4 2
0 4 3
0 4 4
1 0 0
1 0 1
1 0 2
1 0 3
1 0 4
1 1 0
1 1 1
1 1 2
1 1 3
1 1 4
1 2 0
1 2 1
1 2 2
1 2 3
1 2 4
1 3 0
1 3 1
1 3 2
1 3 3
1 3 4
1 4 0
1 4 1
1 4 2
1 4 3
1 4 4
2 0 0
2 0 1
2 0 2
2 0 3
2 0 4
2 1 0
2 1 1
2 1 2
2 1 3
2 1 4
2 2 0
2 2 1
2 2 2
2 2 3
2 2 4
2 3 0
2 3 1
2 3 2
2 3 3
2 3 4
2 4 0
2 4 1
2 4 2
2 4 3
2 4 4
3 0 0
3 0 1
3 0 2
3 0 3
3 0 4
3 1 0
3 1 1
3 1 2
3 1 3
3 1 4
3 2 0
3 2 1
3 2 2
3 2 3
3 2 4
3 3 0
3 3 1
3 3 2
3 3 3
3 3 4
3 4 0
3 4 1
3 4 2
3 4 3
3 4 4
4 0 0
4 0 1
4 0 2
4 0 3
4 0 4
4 1 0
4 1 1
4 1 2
4 1 3
4 1 4
4 2 0
4 2 1
4 2 2
4 2 3
4 2 4
4 3 0
4 3 1
4 3 2
4 3 3
4 3 4
4 4 0
4 4 1
4 4 2
4 4 3
4 4 4


In [256]:
constraints=[(0,1), (0,1), (0,1)]
steps=[1,1,1]

currVars = [0,0,0]
varIndex = 0
while(varIndex != len(steps)):
    print(currVars, varIndex)
    
    if(currVars[varIndex] > constraints[varIndex][1]):
        currVars[varIndex] = constraints[varIndex][0]
        varIndex = varIndex + 1
        if(varIndex < len(steps)):
            currVars[varIndex] = currVars[varIndex] + steps[varIndex]
    else:
        varIndex = 0
        currVars[varIndex] = currVars[varIndex] + steps[varIndex]

[0, 0, 0] 0
[1, 0, 0] 0
[2, 0, 0] 0
[0, 1, 0] 1
[1, 1, 0] 0
[2, 1, 0] 0
[0, 2, 0] 1
[0, 0, 1] 2
[1, 0, 1] 0
[2, 0, 1] 0
[0, 1, 1] 1
[1, 1, 1] 0
[2, 1, 1] 0
[0, 2, 1] 1
[0, 0, 2] 2
